## PEFT 进阶操作

### 1、自定义模型适配

In [1]:
import torch
from torch import nn
from peft import LoraConfig, get_peft_model, PeftModel

In [2]:
net1 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net1

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [3]:
for name, para in net1.named_parameters():
    print(name)

0.weight
0.bias
2.weight
2.bias


In [4]:
config1 = LoraConfig(target_modules=["0"])

In [5]:
model1 = get_peft_model(net1, config1)
model1

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

### 2、多适配器加载与切换

In [6]:
model1.save_pretrained("lora1")

In [7]:
config2 = LoraConfig(target_modules=["2"])
config2

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'2'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [8]:
net1

Sequential(
  (0): lora.Linear(
    (base_layer): Linear(in_features=10, out_features=10, bias=True)
    (lora_dropout): ModuleDict(
      (default): Identity()
    )
    (lora_A): ModuleDict(
      (default): Linear(in_features=10, out_features=8, bias=False)
    )
    (lora_B): ModuleDict(
      (default): Linear(in_features=8, out_features=10, bias=False)
    )
    (lora_embedding_A): ParameterDict()
    (lora_embedding_B): ParameterDict()
    (lora_magnitude_vector): ModuleDict()
  )
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [9]:
model2 = get_peft_model(net1, config2)
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=2, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=2, bia

In [10]:
model2.save_pretrained("lora2")

In [15]:
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net2

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [16]:
model = PeftModel.from_pretrained(net2, model_id="lora1", adapter_name="lora1")
model

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (lora1): Identity()
        )
        (lora_A): ModuleDict(
          (lora1): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (lora1): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

In [17]:
model.load_adapter("lora2", adapter_name="lora2")
model

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (lora1): Identity()
        )
        (lora_A): ModuleDict(
          (lora1): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (lora1): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=2, bias=True)
        (lora_dropout): ModuleDict(
          (lora2): Identity()
        )
        (lora_A): ModuleDict(
          (lora2): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (lora2): Linear(in_features=8, out_features=2, bias=False)
   

In [18]:
model.active_adapter

'lora1'

In [19]:
model(torch.arange(0, 10).view(1, 10).float())

tensor([[-1.9952,  1.1457]], grad_fn=<AddmmBackward0>)

In [20]:
for name, param in model.named_parameters():
    print(name, param)

base_model.model.0.base_layer.weight Parameter containing:
tensor([[-0.0414, -0.1651,  0.2459, -0.1529,  0.2911, -0.0136,  0.2096, -0.0513,
          0.0600, -0.0765],
        [ 0.1274,  0.2460,  0.2658,  0.2475,  0.1282,  0.1452, -0.3047,  0.0442,
          0.2055,  0.0526],
        [ 0.1313, -0.2728,  0.2724, -0.1882,  0.2271,  0.3057,  0.2635, -0.2568,
         -0.2739,  0.0197],
        [ 0.3041,  0.2124, -0.1669, -0.1543, -0.2298, -0.0651, -0.1738,  0.0727,
         -0.1295,  0.2249],
        [-0.0296,  0.0120,  0.2147, -0.0565, -0.1278,  0.0752, -0.0741,  0.1854,
         -0.1090, -0.1683],
        [ 0.2565,  0.2665,  0.2329,  0.2191,  0.0364, -0.0440, -0.1142, -0.1982,
         -0.1645,  0.1259],
        [ 0.1487, -0.1933, -0.2263,  0.0323,  0.2159,  0.0452, -0.0217,  0.2404,
          0.1106,  0.2523],
        [ 0.1606,  0.2354, -0.0365, -0.1610,  0.2193, -0.0841, -0.2802, -0.2409,
         -0.3083,  0.1488],
        [-0.1851,  0.0918,  0.1800,  0.0834, -0.0813, -0.1944,  0.267

In [23]:
for name, param in model.named_parameters():
    if name in ["base_model.model.0.lora_A.lora1.weight", "base_model.model.0.lora_B.lora1.weight"]:
        param.data = torch.ones_like(param)

In [24]:
model(torch.arange(0, 10).view(1, 10).float())

tensor([[-364.4426, -253.1330]], grad_fn=<AddmmBackward0>)

In [25]:
model.set_adapter("lora2")

In [27]:
model.active_adapters

['lora2']

In [28]:
model(torch.arange(0, 10).view(1, 10).float())

tensor([[-1.9952,  1.1457]], grad_fn=<AddBackward0>)

### 3、禁用适配器

In [29]:
model.set_adapter("lora1")

In [30]:
model(torch.arange(0, 10).view(1, 10).float())

tensor([[-364.4426, -253.1330]], grad_fn=<AddmmBackward0>)

In [31]:
with model.disable_adapter():
    print(model(torch.arange(0, 10).view(1, 10).float()))

tensor([[-1.9952,  1.1457]])
